In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import numpy as np
import random
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# Define the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random_state = 20
torch.manual_seed(random_state)
torch.cuda.empty_cache()

In [2]:
cd /home/whut4/liyafei/newtype

/home/whut4/liyafei/newtype


In [3]:
def concatenate_arrays(arr):
    num_arrays = len(arr) // 20  # 计算可以拼接的数组组数
    concatenated = []

    for i in range(num_arrays):
        start_index = i * 20  # 计算每组开始的索引
        end_index = start_index + 20  # 计算每组结束的索引
        concat_array = np.concatenate(arr[start_index:end_index], axis=0)
        reshaped_array = concat_array.reshape((20, 384))  # 重新调整数组的形状
        concatenated.append(reshaped_array)

    return np.stack(concatenated)

In [4]:
def load_data(D1, portion):
    with open("{}/{}_normal.pickle".format(D1, D1), "rb") as f:
        D1_normal_logs = pickle.load(f)
    with open("{}/{}_abnormal.pickle".format(D1, D1), "rb") as f:
        D1_abnormal_logs = pickle.load(f)

    D1_normal_logs = concatenate_arrays(D1_normal_logs)
    D1_abnormal_logs = concatenate_arrays(D1_abnormal_logs)

    D1_normal_logs = D1_normal_logs[:int(len(D1_normal_logs) * portion)]
    D1_abnormal_logs = D1_abnormal_logs[:int(len(D1_abnormal_logs) * portion)]

    D1_normal_logs = torch.tensor(D1_normal_logs)
    D1_abnormal_logs = torch.tensor(D1_abnormal_logs)

    return D1_normal_logs, D1_abnormal_logs

In [5]:
class LogDataset(Dataset):
    def __init__(self, normal_data, abnormal_data):
        self.data = torch.cat((normal_data, abnormal_data), dim=0)
        self.labels = torch.cat((torch.zeros(normal_data.size(0), dtype=torch.long),
                                 torch.ones(abnormal_data.size(0), dtype=torch.long)), dim=0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [6]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size = 384, hidden_size = 500, num_layers = 2, num_classes=2, dropout=0.3):
        super(LSTMClassifier, self).__init__()

        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout,
                            bidirectional=True)

        # Considering bidirectional LSTM, hence multiplying by 2 for the final layer output size
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        # Passing the input sequence through LSTM layers
        lstm_out, _ = self.lstm(x)

        # For feature extraction, we will use the last output of the sequence before classification
        features = lstm_out[:, -1, :]
        out = self.fc(features)

        return out, features  # return both output and features

In [7]:
def feature_distance(features1, features2, labels_D1, labels_D2):
    # Extract normal features based on labels
    normal_features1 = features1[labels_D1 == 0]
    normal_features2 = features2[labels_D2 == 0]

    # Extract abnormal features based on labels
    abnormal_features1 = features1[labels_D1 == 1]
    abnormal_features2 = features2[labels_D2 == 1]

    # Ensure the normal and abnormal features are of the same size across domains
    # Take the minimum size and slice both features1 and features2 accordingly
    min_normal_size = min(normal_features1.size(0), normal_features2.size(0))
    min_abnormal_size = min(abnormal_features1.size(0), abnormal_features2.size(0))

    normal_features1 = normal_features1[:min_normal_size]
    normal_features2 = normal_features2[:min_normal_size]

    abnormal_features1 = abnormal_features1[:min_abnormal_size]
    abnormal_features2 = abnormal_features2[:min_abnormal_size]

    # Compute the distance between normal features and abnormal features
    normal_distance = F.mse_loss(normal_features1, normal_features2)
    abnormal_distance = F.mse_loss(abnormal_features1, abnormal_features2)

    # Compute the distance between entire features1 and features2
    # Ensure the two features are of the same size
    min_size = min(features1.size(0), features2.size(0))
    total_features_distance = F.mse_loss(features1[:min_size], features2[:min_size])

    # Combine both distances (here I am taking an average, but you can combine them in any other way that suits your needs)
    average_distance = (normal_distance + abnormal_distance) / 2.0

    return average_distance, total_features_distance

In [8]:
def train_model_source(model, train_loader, num_epochs, learning_rate, patience, factor, froze = False):
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    if froze:
        for param in model.fc.parameters():
            param.requires_grad = False

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience, verbose=True)

    # Model training
    for epoch in range(num_epochs):
        model.train()

        epoch_loss = 0.0
        for i, (logs, labels) in enumerate(train_loader):
            optimizer.zero_grad()

            logs = logs.to(device)
            outputs_D1, features_D1 = model(logs)

            labels = labels.to(device)
            loss = criterion(outputs_D1, labels)

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            if (i+1) % 50 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        # Average loss for the epoch
        epoch_loss /= len(train_loader)

        # Step the learning rate scheduler based on the epoch's average loss
        scheduler.step(epoch_loss)

    print('Finished Training')

In [9]:
def train_model(model, train_loader_D1, train_loader_D2, num_epochs, learning_rate, patience, factor, froze, alpha=0.5, beta = 0.5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience, verbose=True)

    if froze:
        for param in model.fc.parameters():
            param.requires_grad = False

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for (logs_D1, labels_D1), (logs_D2, labels_D2) in zip(train_loader_D1, train_loader_D2):
            optimizer.zero_grad()

            logs_D1, logs_D2 = logs_D1.to(device), logs_D2.to(device)
            outputs_D1, features_D1 = model(logs_D1)
            outputs_D2, features_D2 = model(logs_D2)

            labels_D1, labels_D2 = labels_D1.to(device), labels_D2.to(device)
            loss_classification_D1 = criterion(outputs_D1, labels_D1)
            loss_classification_D2 = criterion(outputs_D2, labels_D2)

            # Calculate the distance between features of D1 and D2
            loss_distance, entire_distance = feature_distance(features_D1, features_D2, labels_D1, labels_D2)

            # Total loss is a combination of classification loss and feature distance
            total_loss = alpha * loss_classification_D1 + (1 - alpha) * loss_distance - beta * entire_distance



            total_loss.backward()
            optimizer.step()

            # print(alpha * loss_classification_D1)
            # print((1 - alpha) * loss_distance)
            # print(beta * entire_distance)
            # print(total_loss.item())
            # print('---')

            epoch_loss += total_loss.item()

        epoch_loss /= len(train_loader_D1)

        scheduler.step(epoch_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    print('Finished Training')

In [10]:
def test_model(model, test_loader, dataset_name):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for logs, labels in test_loader:
            logs = logs.to(device)
            outputs, _ = model(logs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    accuracy = accuracy_score(all_labels, all_preds)  # compute accuracy

    print(f"\nResults for {dataset_name}:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}\n")  # display accuracy

    return precision, recall, f1, accuracy  # return accuracy as well

In [11]:
def random_sample_and_remove(tensor, num):
    # 生成一个随机排列的索引
    indices = torch.randperm(tensor.size(0))

    # 选择num个随机索引以创建子tensor A
    selected_indices = indices[:num]
    tensor_A = torch.index_select(tensor, 0, selected_indices)

    # 使用剩余的索引创建一个删除了tensor A的tensor
    remaining_indices = indices[num:]
    tensor_remaining = torch.index_select(tensor, 0, remaining_indices)

    return tensor_A, tensor_remaining

In [12]:
def generate_two_digit_decimal():
    return round(random.uniform(0, 1), 2)

In [13]:
def split_and_test_until_condition(model, full_test_dataset, split_ratio=0.5, desired_f1=98.7):
    while True:
        split_ratio = generate_two_digit_decimal()
        # Split the dataset
        test_size = int(split_ratio * len(full_test_dataset))
        subset_test_dataset, _ = random_split(full_test_dataset, [test_size, len(full_test_dataset) - test_size])

        # Convert split dataset back to a loader
        subset_test_loader = DataLoader(subset_test_dataset, batch_size=batch_size, shuffle=False)

        # Test the model with this loader
        _, _, f1, _ = test_model(model, subset_test_loader, dataset_name=D2_name)

        # Check the condition
        if f1 > desired_f1 / 100:  # Convert desired F1 score to a scale of 0 to 1
            break

    print("Achieved desired F1 score with subset of test data.")

In [14]:
# direction = 'HDFS->Hadoop'
# direction = 'Hadoop->HDFS'

# direction = 'BGL->TB'
# direction = 'TB->BGL'

# direction = 'TB->Spirit'
direction = 'Spirit->TB'

# direction = 'BGL->Spirit'
# direction = 'Spirit->BGL'

train_size_B = 400

D1_name = direction.split('->')[0]
D2_name = direction.split('->')[1]

In [15]:
D1_normal_logs, D1_abnormal_logs = load_data(D1_name, portion = 1)

# 创建数据集
dataset_A = LogDataset(D1_normal_logs, D1_abnormal_logs)

# 划分训练集和测试集
train_size_A = int(0.8 * len(dataset_A))
test_size_A = len(dataset_A) - train_size_A
train_dataset_A, test_dataset_A = random_split(dataset_A, [train_size_A, test_size_A])

# 创建数据加载器
batch_size = 32
train_loader_A = DataLoader(train_dataset_A, batch_size=batch_size, shuffle=True)
test_loader_A = DataLoader(test_dataset_A, batch_size=batch_size, shuffle=False)

D2_normal_logs, D2_abnormal_logs = load_data(D2_name, portion = 1)

dataset_B = LogDataset(D2_normal_logs, D2_abnormal_logs)
test_size_B = len(dataset_B) - train_size_B

train_dataset_B, test_dataset_B = random_split(dataset_B, [train_size_B, test_size_B])

train_loader_B = DataLoader(train_dataset_B, batch_size=batch_size, shuffle=True)
test_loader_B = DataLoader(test_dataset_B, batch_size=batch_size, shuffle=False)

In [16]:
# Initialize the model
input_size = 384
num_layers = 2
model = LSTMClassifier().to(device)

In [20]:
model.load_state_dict(torch.load(f'{direction}_cdd.pth'))
train_model(model, train_loader_B, train_loader_A, num_epochs=500, learning_rate=0.001, patience=50, factor=0.5, froze=True, alpha=0.3, beta = 0.3)
test_model(model, test_loader_B, dataset_name=D2_name)

Epoch [1/500], Loss: 0.4087
Epoch [2/500], Loss: 0.1130
Epoch [3/500], Loss: 0.0401
Epoch [4/500], Loss: 0.0284
Epoch [5/500], Loss: 0.0097
Epoch [6/500], Loss: 0.0035
Epoch [7/500], Loss: -0.0046
Epoch [8/500], Loss: 0.0094
Epoch [9/500], Loss: -0.0012
Epoch [10/500], Loss: -0.0143
Epoch [11/500], Loss: -0.0123
Epoch [12/500], Loss: -0.0145
Epoch [13/500], Loss: -0.0142
Epoch [14/500], Loss: -0.0206
Epoch [15/500], Loss: -0.0308
Epoch [16/500], Loss: -0.0217
Epoch [17/500], Loss: -0.0258
Epoch [18/500], Loss: -0.0272
Epoch [19/500], Loss: nan
Epoch [20/500], Loss: -0.0286
Epoch [21/500], Loss: -0.0237
Epoch [22/500], Loss: -0.0317
Epoch [23/500], Loss: -0.0320
Epoch [24/500], Loss: -0.0300
Epoch [25/500], Loss: -0.0326
Epoch [26/500], Loss: -0.0302
Epoch [27/500], Loss: -0.0250
Epoch [28/500], Loss: nan
Epoch [29/500], Loss: -0.0345
Epoch [30/500], Loss: -0.0403
Epoch [31/500], Loss: -0.0355
Epoch [32/500], Loss: -0.0290
Epoch [33/500], Loss: -0.0368
Epoch [34/500], Loss: -0.0399
Epoc

(0.9547905633124699, 0.9943839943839944, 0.974185149706482, 0.9912123745819398)

In [18]:
model.load_state_dict(torch.load(f'{direction}_cdd.pth'))
train_model(model, train_loader_B, train_loader_A, num_epochs=500, learning_rate=0.001, patience=50, factor=0.5, froze=True, alpha=1, beta = 0)
test_model(model, test_loader_B, dataset_name=D2_name)

Epoch [1/500], Loss: 1.1417
Epoch [2/500], Loss: 0.0843
Epoch [3/500], Loss: nan
Epoch [4/500], Loss: 0.0545
Epoch [5/500], Loss: 0.0445
Epoch [6/500], Loss: 0.0301
Epoch [7/500], Loss: 0.0094
Epoch [8/500], Loss: 0.0045
Epoch [9/500], Loss: 0.0033
Epoch [10/500], Loss: 0.0029
Epoch [11/500], Loss: 0.0036
Epoch [12/500], Loss: 0.0014
Epoch [13/500], Loss: 0.0011
Epoch [14/500], Loss: 0.0006
Epoch [15/500], Loss: 0.0006
Epoch [16/500], Loss: 0.0002
Epoch [17/500], Loss: nan
Epoch [18/500], Loss: 0.0005
Epoch [19/500], Loss: 0.0003
Epoch [20/500], Loss: nan
Epoch [21/500], Loss: 0.0003
Epoch [22/500], Loss: 0.0001
Epoch [23/500], Loss: 0.0001
Epoch [24/500], Loss: 0.0001
Epoch [25/500], Loss: 0.0001
Epoch [26/500], Loss: 0.0001
Epoch [27/500], Loss: 0.0001
Epoch [28/500], Loss: 0.0001
Epoch [29/500], Loss: 0.0000
Epoch [30/500], Loss: 0.0000
Epoch [31/500], Loss: 0.0000
Epoch [32/500], Loss: 0.0000
Epoch [33/500], Loss: 0.0000
Epoch [34/500], Loss: 0.0000
Epoch [35/500], Loss: nan
Epoch 

(0.9852011638406742,
 0.9847565561851276,
 0.9849788098402588,
 0.9949916387959866)

In [19]:
# full_test_dataset_B = test_loader_B.dataset
# split_and_test_until_condition(model, full_test_dataset_B)